In [1]:
import sys
import time
import os
import subprocess
import math
import matplotlib.pyplot as plt
import numpy as np
from astropy.table import Table, Column 
from scipy.stats import linregress
from scipy import interpolate
from scipy import polyval, polyfit
from scipy.optimize import curve_fit
from scipy import odr
import pylab as py
from matplotlib import gridspec
import sklearn.datasets as ds
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
import corner
import emcee
import scipy.optimize as op
from scipy.linalg import cholesky, inv,det
from scipy.optimize import minimize
import random
from astropy.table import Table, Column
import astropy.stats as st

from calibTools import *

In [2]:
table   = np.genfromtxt('corrected_mags_linewidth_all.csv' , delimiter='|', 
                        filling_values=-1, names=True, dtype=None, encoding=None)


Au_e  = getReddening_err(table, band1='u', band2='w2')
Ag_e  = getReddening_err(table, band1='g', band2='w2')
Ar_e  = getReddening_err(table, band1='r', band2='w2')
Ai_e  = getReddening_err(table, band1='i', band2='w2')
Az_e  = getReddening_err(table, band1='z', band2='w2')
Aw1_e  = getReddening_err(table, band1='w1', band2='w2')
Aw2_e  = getReddening_err(table, band1='w2', band2='w2')


catal   = np.genfromtxt('ESN_HI_catal_all.csv' , delimiter=',', 
                        filling_values=-1, names=True, dtype=None, encoding=None)
pgc_catal   = catal['pgc']
R50u_catal  = catal['R50_u']
R50g_catal  = catal['R50_g']
R50r_catal  = catal['R50_r']
R50i_catal  = catal['R50_i']
R50z_catal  = catal['R50_z']
R50w1_catal = catal['R50_w1']
R50w2_catal = catal['R50_w2']
Sba_catal   = catal['Sba']
Wba_catal   = catal['Wba']


pgc    = table['pgc']
Name   = table['Name']
inc    = table['inc']
inc_e  = table['inc_e']
Wmx    = table['Wmx']
eWmx   = table['eWmx']
logWimx = table['logWimx']
logWimx_e  = table['logWimx_e']
m21    = table['m21']
m21_e  = table['m21_e']
uu     = table['u']
gg     = table['g']
rr     = table['r']
ii     = table['i']
zz     = table['z']
w1     = table['w1']
w2     = table['w2']


u_     = table['u_']
g_     = table['g_']
r_     = table['r_']
i_     = table['i_']
z_     = table['z_']
w1_    = table['w1_']
w2_    = table['w2_']

N = len(pgc)
R50u  = np.zeros(N)
R50g  = np.zeros(N)
R50r  = np.zeros(N)
R50i  = np.zeros(N)
R50z  = np.zeros(N)
R50w1 = np.zeros(N)
R50w2 = np.zeros(N)
Sba   = np.zeros(N)
Wba   = np.zeros(N)

u_s   = table['u_sss']
g_s   = table['g_sss']
r_s   = table['r_sss']
i_s   = table['i_sss']
z_s   = table['z_sss']
w1_s  = table['w1_sss']
w2_s  = table['w2_sss']


for i, idd in enumerate(pgc):
    if idd in pgc_catal:
        indx,   = np.where(pgc_catal==idd)
        R50u[i]  = R50u_catal[indx][0]
        R50g[i]  = R50g_catal[indx][0]
        R50r[i]  = R50r_catal[indx][0]
        R50i[i]  = R50i_catal[indx][0]
        R50z[i]  = R50z_catal[indx][0]
        R50w1[i] = R50w1_catal[indx][0]
        R50w2[i] = R50w2_catal[indx][0]
        Sba[i]   = Sba_catal[indx][0]
        Wba[i]   = Wba_catal[indx][0]


In [7]:
Nests = []
Cnames = []
PGCs  = [] 

n1 = 0
n2 = 0 
for band in ['w1']:
    
    calib_Clusters = allCluster(table, band=band)

    for key in calib_Clusters:
        cluster = calib_Clusters[key]
        for i, idd in enumerate(cluster['pgc']):
            if not idd in PGCs:
                PGCs.append(idd)
                Nests.append(key)
                Cnames.append(cluster['name'])
                n1+=1
        for i, idd in enumerate(cluster['pgc_w']):
            if not idd in PGCs:
                PGCs.append(idd)
                Nests.append(key)
                Cnames.append(cluster['name'])   
                n2+=1

print '# of galaxies in Clusters: ', len(PGCs)            


n_zp = 0
nzp0 = 0
for band in ['i', 'w1']:
    cluster = all_ZP(table, band=band)
    for i, idd in enumerate(cluster['pgc']):
        if not idd in PGCs:
            PGCs.append(idd)
            Nests.append('ZP')
            Cnames.append('ZP calibrator')
            n_zp += 1
    for i, idd in enumerate(cluster['pgc_w']):
        if not idd in PGCs:
            PGCs.append(idd)
            Nests.append('ZP')
            Cnames.append('ZP calibrator')               
            n_zp += 1
            nzp0+=1


print '# of total galaxies: ', len(PGCs)
print '# of ZP galaxies: ', n_zp


print n1,n2,nzp0

# of galaxies in Clusters:  619
# of total galaxies:  706
# of ZP galaxies:  87
600 19 5


In [ ]:

indx = np.argsort(PGCs)

Nrow = 0
for i in indx:
    
    row = ''
    idd = PGCs[i]
    
    j, = np.where(pgc == idd)
    j = j[0]
    
    if Cnames[i].strip()=='Virgo':
        Nrow+=1
        
        row += str(idd) + ' & '
        row += Name[j].strip()  + ' & '
        
        row += '%.0f'%inc[j] + '$\pm$' + '%.0f'%inc_e[j] + ' & '
        row += '%.0f'%Wmx[j] + '$\pm$' + '%.0f'%eWmx[j] + ' & '
        row += '%.3f'%logWimx[j] + '$\pm$' + '%.3f'%logWimx_e[j] + ' & '
        row += '%.3f'%m21[j] + '$\pm$' + '%.3f'%m21_e[j] + ' & '
        
        row += '%.2f'%uu[j] + ' & '
        row += '%.2f'%gg[j] + ' & '
        row += '%.2f'%rr[j] + ' & '
        row += '%.2f'%ii[j] + ' & '
        row += '%.2f'%zz[j] + ' & '
        row += '%.2f'%w1[j] + ' & '
        row += '%.2f'%w2[j] + ' & '
        
        row += '%.2f'%Sba[j] + ' & '
        row += '%.2f'%Wba[j] + ' & '

        row += '%.2f'%R50u[j] + ' & '
        row += '%.2f'%R50g[j] + ' & '
        row += '%.2f'%R50r[j] + ' & '
        row += '%.2f'%R50i[j] + ' & '
        row += '%.2f'%R50z[j] + ' & '
        row += '%.2f'%R50w1[j] + ' & '
        row += '%.2f'%R50w2[j] + ' \\\\ '        
        
        
        print row
        
    
    if Nrow==15:
        break
    

In [ ]:

indx = np.argsort(PGCs)

Nrow = 0
for i in indx:
    
    row = ''
    idd = PGCs[i]
    
    j, = np.where(pgc == idd)
    j = j[0]
    
    if Cnames[i].strip()=='Virgo':
        Nrow+=1
        
        row += str(idd) + ' & '
        row += Cnames[i].strip()  + ' & '
               
        row += '%.2f'%u_[j]  + ' & '
        row += '%.2f'%g_[j]  + ' & '
        row += '%.2f'%r_[j]  + ' & '
        row += '%.2f'%i_[j]  + ' & '
        row += '%.2f'%z_[j]  + ' & '
        row += '%.2f'%w1_[j] + ' & '
        row += '%.2f'%w2_[j] + ' & '
        
        
        row += '%.2f'%(u_[j]-u_s[j]) + '$\pm$' + '%.2f'%Au_e[j]  + ' & '
        row += '%.2f'%(g_[j]-g_s[j]) + '$\pm$' + '%.2f'%Ag_e[j]  + ' & '
        row += '%.2f'%(r_[j]-r_s[j]) + '$\pm$' + '%.2f'%Ar_e[j]  + ' & '
        row += '%.2f'%(i_[j]-i_s[j]) + '$\pm$' + '%.2f'%Ai_e[j]  + ' & '
        row += '%.2f'%(z_[j]-z_s[j]) + '$\pm$' + '%.2f'%Az_e[j]  + ' & '
        row += '%.2f'%(w1_[j]-w1_s[j]) + '$\pm$' + '%.2f'%Aw1_e[j] + ' & '
        row += '%.2f'%(w2_[j]-w2_s[j]) + '$\pm$' + '%.2f'%Aw2_e[j] + ' & '        
        
        
        row += '%.2f'%u_s[j]  + ' & '
        row += '%.2f'%g_s[j]  + ' & '
        row += '%.2f'%r_s[j]  + ' & '
        row += '%.2f'%i_s[j]  + ' & '
        row += '%.2f'%z_s[j]  + ' & '
        row += '%.2f'%w1_s[j] + ' & '
        row += '%.2f'%w2_s[j] + ' \\\\ '        
      
        
        
        print row
        
    
    if Nrow==15:
        break
    

In [ ]:

indx = np.argsort(PGCs)

Nrow = 0

with open("Table1.csv", 'w') as file:
    
    
    first_row = 'pgc, name, inc, inc_e, Wmx, Wmx_e, logWimx, logWimx_e, m21, m21_e, u, g, r, i, z, w1, w2, Sba, Wba, R50u, R50g, R50r, R50i, R50z, R50w1, R50w2, ClusteName, u_, g_, r_, i_, z_, w1_, w2_, Au, Au_e, Ag, Ag_e, Ar, Ar_e, Ai, Ai_e, Az, Az_e, Aw1, Aw1_e, Aw2, Aw2_e, us, gs, rs, is, zs, w1s, w2s'
    file.write(first_row+' \n')

    for i in indx:

        row = ''
        idd = PGCs[i]

        j, = np.where(pgc == idd)
        j = j[0]

        if True: # Cnames[i].strip()=='Virgo':
            Nrow+=1

            row += str(idd) + ' , '
            row += Name[j].strip()  + ' , '

            row += '%.0f'%inc[j] + ', ' + '%.0f'%inc_e[j] + ' , '
            row += '%.0f'%Wmx[j] + ', ' + '%.0f'%eWmx[j] + ' , '
            row += '%.3f'%logWimx[j] + ', ' + '%.3f'%logWimx_e[j] + ' , '
            row += '%.3f'%m21[j] + ', ' + '%.3f'%m21_e[j] + ' , '

            row += '%.2f'%uu[j] + ' , '
            row += '%.2f'%gg[j] + ' , '
            row += '%.2f'%rr[j] + ' , '
            row += '%.2f'%ii[j] + ' , '
            row += '%.2f'%zz[j] + ' , '
            row += '%.2f'%w1[j] + ' , '
            row += '%.2f'%w2[j] + ' , '

            row += '%.2f'%Sba[j] + ' , '
            row += '%.2f'%Wba[j] + ' , '

            row += '%.2f'%R50u[j] + ' , '
            row += '%.2f'%R50g[j] + ' , '
            row += '%.2f'%R50r[j] + ' , '
            row += '%.2f'%R50i[j] + ' , '
            row += '%.2f'%R50z[j] + ' , '
            row += '%.2f'%R50w1[j] + ' , '
            row += '%.2f'%R50w2[j] + ' , '


            row += Cnames[i].strip()  + ' , '

            row += '%.2f'%u_[j]  + ' , '
            row += '%.2f'%g_[j]  + ' , '
            row += '%.2f'%r_[j]  + ' , '
            row += '%.2f'%i_[j]  + ' , '
            row += '%.2f'%z_[j]  + ' , '
            row += '%.2f'%w1_[j] + ' , '
            row += '%.2f'%w2_[j] + ' , '


            row += '%.2f'%(u_[j]-u_s[j]) + ', ' + '%.2f'%Au_e[j]  + ' , '
            row += '%.2f'%(g_[j]-g_s[j]) + ', ' + '%.2f'%Ag_e[j]  + ' , '
            row += '%.2f'%(r_[j]-r_s[j]) + ', ' + '%.2f'%Ar_e[j]  + ' , '
            row += '%.2f'%(i_[j]-i_s[j]) + ', ' + '%.2f'%Ai_e[j]  + ' , '
            row += '%.2f'%(z_[j]-z_s[j]) + ', ' + '%.2f'%Az_e[j]  + ' , '
            row += '%.2f'%(w1_[j]-w1_s[j]) + ', ' + '%.2f'%Aw1_e[j] + ' , '
            row += '%.2f'%(w2_[j]-w2_s[j]) + ', ' + '%.2f'%Aw2_e[j] + ' , '        


            row += '%.2f'%u_s[j]  + ' , '
            row += '%.2f'%g_s[j]  + ' , '
            row += '%.2f'%r_s[j]  + ' , '
            row += '%.2f'%i_s[j]  + ' , '
            row += '%.2f'%z_s[j]  + ' , '
            row += '%.2f'%w1_s[j] + ' , '
            row += '%.2f'%w2_s[j] + ' \n '



            #print row
            file.write(row)
        
    
#     if Nrow==15:
#         break  


In [ ]:
table   = np.genfromtxt('Table1.csv' , delimiter=',', 
                        filling_values=-1, names=True, dtype=None, encoding=None)

myTable = Table()
myTable.add_column(Column(data=table['pgc'], name='PGC'))
myTable.add_column(Column(data=table['name'], name='Name'))
myTable.add_column(Column(data=table['inc'], name='inc'))
myTable.add_column(Column(data=table['inc_e'], name='inc_e'))
myTable.add_column(Column(data=table['Wmx'], name='Wmx'))
myTable.add_column(Column(data=table['Wmx_e'], name='Wmx_e'))
myTable.add_column(Column(data=table['logWimx'], name='logWimx', format='%0.3f'))
myTable.add_column(Column(data=table['logWimx_e'], name='logWimx_e', format='%0.3f')) 
myTable.add_column(Column(data=table['m21'], name='m21', format='%0.3f'))
myTable.add_column(Column(data=table['m21_e'], name='m21_e', format='%0.3f'))

myTable.add_column(Column(data=table['u'], name='u', format='%0.2f'))
myTable.add_column(Column(data=table['g'], name='g', format='%0.2f'))
myTable.add_column(Column(data=table['r'], name='r', format='%0.2f'))
myTable.add_column(Column(data=table['i'], name='i', format='%0.2f'))
myTable.add_column(Column(data=table['z'], name='z', format='%0.2f'))
myTable.add_column(Column(data=table['w1'], name='w1', format='%0.2f'))
myTable.add_column(Column(data=table['w2'], name='w2', format='%0.2f'))

myTable.add_column(Column(data=table['Sba'], name='Sba', format='%0.2f'))
myTable.add_column(Column(data=table['Wba'], name='Wba', format='%0.2f'))

myTable.add_column(Column(data=table['R50u'], name='R50_u', format='%0.2f'))
myTable.add_column(Column(data=table['R50g'], name='R50_g', format='%0.2f'))
myTable.add_column(Column(data=table['R50r'], name='R50_r', format='%0.2f'))
myTable.add_column(Column(data=table['R50i'], name='R50_i', format='%0.2f'))
myTable.add_column(Column(data=table['R50z'], name='R50_z', format='%0.2f'))
myTable.add_column(Column(data=table['R50w1'], name='R50_w1', format='%0.2f'))
myTable.add_column(Column(data=table['R50w2'], name='R50_w2', format='%0.2f'))

myTable.add_column(Column(data=table['ClusteName'], name='ClusteName'))

myTable.add_column(Column(data=table['u_'], name='u_', format='%0.2f'))
myTable.add_column(Column(data=table['g_'], name='g_', format='%0.2f'))
myTable.add_column(Column(data=table['r_'], name='r_', format='%0.2f'))
myTable.add_column(Column(data=table['i_'], name='i_', format='%0.2f'))
myTable.add_column(Column(data=table['z_'], name='z_', format='%0.2f'))
myTable.add_column(Column(data=table['w1_'], name='w1_', format='%0.2f'))
myTable.add_column(Column(data=table['w2_'], name='w2_', format='%0.2f'))

myTable.add_column(Column(data=table['Au'], name='Au', format='%0.2f'))
myTable.add_column(Column(data=table['Au_e'], name='Au_e', format='%0.2f'))
myTable.add_column(Column(data=table['Ag'], name='Ag', format='%0.2f'))
myTable.add_column(Column(data=table['Ag_e'], name='Ag_e', format='%0.2f'))
myTable.add_column(Column(data=table['Ar'], name='Ar', format='%0.2f'))
myTable.add_column(Column(data=table['Ar_e'], name='Ar_e', format='%0.2f'))
myTable.add_column(Column(data=table['Ai'], name='Ai', format='%0.2f'))
myTable.add_column(Column(data=table['Ai_e'], name='Ai_e', format='%0.2f'))
myTable.add_column(Column(data=table['Az'], name='Az', format='%0.2f'))
myTable.add_column(Column(data=table['Az_e'], name='Az_e', format='%0.2f'))
myTable.add_column(Column(data=table['Aw1'], name='Aw1', format='%0.2f'))
myTable.add_column(Column(data=table['Aw1_e'], name='Aw1_e', format='%0.2f'))
myTable.add_column(Column(data=table['Aw2'], name='Aw2', format='%0.2f'))
myTable.add_column(Column(data=table['Aw2_e'], name='Aw2_e', format='%0.2f'))

myTable.add_column(Column(data=table['us'], name='u_s', format='%0.2f'))
myTable.add_column(Column(data=table['gs'], name='g_s', format='%0.2f'))
myTable.add_column(Column(data=table['rs'], name='r_s', format='%0.2f'))
myTable.add_column(Column(data=table['is'], name='i_s', format='%0.2f'))
myTable.add_column(Column(data=table['zs'], name='z_s', format='%0.2f'))
myTable.add_column(Column(data=table['w1s'], name='w1_s', format='%0.2f'))
myTable.add_column(Column(data=table['w2s'], name='w2_s', format='%0.2f'))

myTable.write('Table1.aligned', format='ascii.fixed_width',delimiter='|', bookend=False, overwrite=True) 
